In [1]:
%load_ext autoreload

In [63]:
%autoreload 

import sys
sys.path.append('../../..')
import pickle

import numpy as np
from tqdm import tqdm_notebook as tqdm

import matplotlib
matplotlib.rc_file('../../matplotlibrc')
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm 
import matplotlib.colors as colors
%matplotlib inline

import darkhistory.physics as phys
import main
import config


Bad key text.latex.preview in file ../../matplotlibrc, line 10 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution


In [59]:
config.test(end_rs=200, iter=2, std_only=True)
# config.test()

Running main.evolve(...): 
******************************************
Testing solution with no DM: 


  0%|          | 0/170.0 [00:00<?, ?it/s]

/home/hongwanl/DarkHistory/examples/DH_Upgrade/New_Results/../../../darkhistory/spec/spectrum.py:861: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)


  0%|          | 0/170.0 [00:00<?, ?it/s]

The maximum relative change in xHI and xHeI is:  6.921374715473938e-06
The maximum relative change in Tm is:  9.590620630994806e-09
The maximum relative change in f_(H ion) is:  1.1690858832125173e-06
The maximum relative change in f_(H ion) is:  1.1690858832125173e-06
The maximum relative change in f_(He ion) is:  1.004167236029474e-06
The maximum relative change in f_(Lya) is:  1.4561289933823218e-06
The maximum relative change in f_(heat) is:  1.8882124488417418e-06
The maximum relative change in f_(cont) is:  4.807906637260828e-07
The maximum relative change in the MLA parameters is:  5.416912616129643e-06
Pickled solution with no DM!
Test complete!


In [ ]:
test_data = pickle.load(open('/scratch/gpfs/hongwanl/DarkHistory/data/std_test_data.p', 'rb'))

ref_data = pickle.load(open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220818_std_result_n_10_high_rs_1555_coarsen_16_reion_False_rtol_1e-6.p', 'rb'))

In [ ]:
print(test_data['MLA'][1:])
print(ref_data['MLA'][1:])

In [61]:
options_dict = {
    'primary':'elec_delta', 'DM_process':'decay', 'mDM':1e8, 'lifetime':3e25,
    'start_rs': 3000, 'high_rs': 1.555e3, 'end_rs':4,
    'reion_switch':True, 'reion_method':'Puchwein', 'heat_switch':True,
    'coarsen_factor':16, 'distort':True, 'fexc_switch': True, 
    'MLA_funcs':None,
    'reprocess_distortion':True, 'nmax':10, 'rtol':1e-6, 'use_tqdm':True, 'iterations':2
}

options_dict_std = {
    'primary':'elec_delta', 'DM_process':'decay', 'mDM':1e8, 'lifetime':3e40,
    'start_rs': 3000, 'high_rs': 1.555e3, 'end_rs':2000,
    'reion_switch':False, 'reion_method':'Puchwein', 'heat_switch':True,
    'coarsen_factor':16, 'distort':True, 'fexc_switch': True, 
    'MLA_funcs':None,
    'reprocess_distortion':True, 'nmax':100, 'rtol':1e-6, 'use_tqdm':True, 'iterations':2
}

In [64]:
%%prun 
# res = main.evolve(**options_dict)
res_std = main.evolve(**options_dict_std)

  0%|          | 0/26.0 [00:00<?, ?it/s]

/home/hongwanl/DarkHistory/examples/DH_Upgrade/New_Results/../../../darkhistory/spec/spectrum.py:861: RuntimeWarning: The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.
  warnings.warn("The new abscissa lies below the old one: only bins that lie within the new abscissa will be rebinned, bins above the abscissa will be discarded.", RuntimeWarning)


  0%|          | 0/26.0 [00:00<?, ?it/s]

         27357173 function calls (27304962 primitive calls) in 113.403 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52   24.873    0.478   86.354    1.661 atomic.py:642(process_MLA)
       52   14.073    0.271   29.479    0.567 atomic.py:226(populate_bound_bound)
       52    7.589    0.146    7.589    0.146 {method 'nonzero' of 'numpy.ndarray' objects}
      156    6.674    0.043    6.674    0.043 {built-in method scipy.sparse.linalg.dsolve._superlu.gssv}
        2    4.046    2.023    7.840    3.920 atomic.py:189(populate_radial)
  1940400    3.566    0.000    3.566    0.000 atomic.py:163(Hey_A)
1609189/1575889    3.128    0.000   13.610    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
   585888    3.111    0.000    3.111    0.000 {method 'reduce' of 'numpy.ufunc' objects}
     5200    2.993    0.001    4.208    0.001 bound_free.py:810(net_spec_n)
      850    2.172    0.003    2.

In [ ]:
plt.figure()

plt.loglog()

res_rs = res[-1]['MLA'][0]
x_std = phys.x_std(res_rs)
Tm_std = phys.Tm_std(res_rs)

C_times_alpha_B = phys.peebles_C(x_std, res_rs) * phys.alpha_recomb(Tm_std, 'HI')


# plt.plot(res['MLA'][0], res['MLA'][1,:], label=r'DM $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B,\mathrm{std}})$')
plt.plot(res_std[-1]['MLA'][0], res_std[-1]['MLA'][1,:] / C_times_alpha_B, label=r'Standard $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B, \mathrm{std}})$')
# plt.plot(res_std_2['MLA'][0], res_std_2['MLA'][1,:] / C_times_alpha_B, label=r'Standard $\tilde{\alpha}_\mathrm{B} / (C_\mathrm{std} \alpha_{B, \mathrm{std}})$')
# plt.plot(res_std['MLA'][0], res_std['MLA'][1,:] / res_std_2['MLA'][1,:])
# plt.plot(res_rs, C_times_alpha_B, 'k--', label=r'$C_\mathrm{std} \alpha_{B, \mathrm{std}}$')


plt.xlabel(r'$1+z$')
plt.ylabel(r'Rate [cm$^{3}$ s$^{-1}$]')
plt.legend()

# plt.xlim(1e3, 3e3)

In [ ]:
plt.figure()

plt.loglog()

plt.plot(res_std['rs'], res_std['x'][:,0] / phys.x_std(res_std['rs']))

plt.plot(res['rs'], res['x'][:,0] / phys.x_std(res['rs']))


# plt.xlim(20, 3000)
# plt.ylim(0.975, 1.025)

In [ ]:
pickle.dump(res_std, open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220822_std_result_n_10_high_rs_1555_coarsen_16_reion_False_rtol_1e-6_iter_2.p', 'wb'))

pickle.dump(res, open('/scratch/gpfs/hongwanl/DarkHistory/data/reference_20220822_mDM_1e8_elec_delta_decay_3e25_n_10_high_rs_1555_coarsen_16_reion_True_rtol_1e-6_iter_2.p', 'wb'))


In [ ]:
nmax = 10

eng_norm_levels = np.divide(
        1., np.arange(nmax+1.)**2, out=np.ones(nmax+1)*np.nan, 
        where=np.arange(nmax+1) != 0
    )
Ennp_mat = (eng_norm_levels[None,:] - eng_norm_levels[:,None]) 

In [ ]:
Ennp_mat[1]

In [ ]:
1/20**2 - 1/1**2

In [ ]:
np.exp(-np.inf/3)